# Testing Easy Torch

In [ ]:
import sys

In [ ]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # View the path and verify

In [ ]:
import easy_data, easy_exp, easy_torch

## Configuration loading

In [ ]:
cfg = easy_exp.cfg.load_configuration("config_nn")

In [ ]:
cfg

In [ ]:
data = easy_data.data.load_data(**cfg["data"])

In [ ]:
loaders = easy_torch.preparation.prepare_data_loaders(data, cfg["model"]["loader_params"])

In [ ]:
cfg["model"]["in_channels"] = data["train_x"].shape[1]
cfg["model"]["out_features"] = data["train_y"].shape[1]

In [ ]:
main_module = easy_torch.model.get_torchvision_model(**cfg["model"])

In [ ]:
exp_found, experiment_id = easy_exp.exp.get_experiment_id(cfg)

In [ ]:
print(exp_found, experiment_id)

In [ ]:
# Set experiment_id in trainer_params
trainer_params = easy_torch.preparation.prepare_experiment_id(cfg["model"]["trainer_params"], experiment_id)

In [ ]:
trainer_params["callbacks"] = easy_torch.preparation.prepare_callbacks(trainer_params)
trainer_params["logger"] = easy_torch.preparation.prepare_logger(trainer_params)
trainer = easy_torch.preparation.prepare_trainer(**trainer_params)

# callbacks = easy_torch.preparation.prepare_callbacks(cfg["model"]["trainer_params"])
# logger = easy_torch.preparation.prepare_logger(cfg["model"]["trainer_params"])
# already_defined = ["callbacks","logger"]
# trainer = easy_torch.preparation.prepare_trainer(**{k:cfg["model"]["trainer_params"][k] for k in cfg["model"]["trainer_params"] if k not in already_defined}, callbacks=callbacks, logger=logger)

In [ ]:
loss = easy_torch.preparation.prepare_loss(cfg["model"]["loss"])

In [ ]:
optimizer = easy_torch.preparation.prepare_optimizer(**cfg["model"]["optimizer"])

In [ ]:
model = easy_torch.process.create_model(main_module, loss, optimizer)

In [ ]:
easy_torch.process.train_model(trainer, model, loaders)

In [ ]:
easy_torch.process.test_model(trainer, model, loaders)

In [ ]:
easy_exp.exp.save_experiment(cfg)